# Introduction to Multi-Armed Bandits——03 Bayesian Bandits and Thompson Sampling

## 参考资料

1. Slivkins A. Introduction to multi-armed bandits[J]. Foundations and Trends® in Machine Learning, 2019, 12(1-2): 1-286.

2. [Thompson Sampling](https://towardsdatascience.com/thompson-sampling-fc28817eacb8)

3. [WhatIThinkAbout/BabyRobot](https://github.com/WhatIThinkAbout/BabyRobot/tree/master/Multi_Armed_Bandits)

[Bandit算法学习[网站优化]](https://blog.csdn.net/weixin_47692652/article/details/128539899)偏实战，而本专栏偏理论学习。

`Bayesian bandit problem` 在 `stochastic bandits` 上增加了 `Bayesian assumption` ：问题实例 $\mathcal{I}$ 最初是从某个已知分布 $\mathbb{P}$ 中抽取的。时间跨度 $T$ 和臂数量 $K$ 是固定的。然后，`stochastic bandits` 的实例由平均奖励向量 $\mu\in [0,1]^K$ 和奖励分布 $(\mathcal{D}_a:\, a\in [K])$ 指定。分布 $\mathbb{P}$ 被称为先验分布，或贝叶斯先验（`Bayesian prior`）。目标是优化 `Bayesian regret` : 

$$
\begin{align}
BR(T):= \mathbb{E}_{\mathcal{I}\sim \mathbb{P}}\left[{ \mathbb{E}\left[{ R(T)\ |\ \mathcal{I} }\ \right]}\ \right]= \mathbb{E}_{\mathcal{I}\sim\mathbb{P}}\left[{\mu^* \cdot T -  \sum_{t\in [T]} \mu(a_t)}\right] \tag{3.1}
\end{align}
$$

`Bayesian bandits`遵循一个著名的`Bayesian statistics`方法：**假设未知量是从一个已知分布中取样的，并对这个分布进行期望优化**。

**简化**：为了简化表述，我们做了几个假设。

1. 首先，已实现的奖励来自于一个分布的`single-parameter family`。有一个实值分布系列 $(\mathcal{D}_\mathcal{v}$, $\mathcal{v} \in[0,1])$，是固定的，也是算法所知道的，每个分布 $\mathcal{D}_\mathcal{v}$ 都有期望值 $\mathcal{v}$ 。典型的例子是**伯努利奖励**和**单位方差高斯分布**。每个手臂 $a$ 的奖励是从分布 $\mathcal{D}_{\mu(a)}$ 中抽取的，其中 ${\mu(a)}\in[0,1]$ 是平均奖励。问题实例完全由 $[0,1]^K$ 中的平均奖励向量 $\mu$ 指定，先验 $\mathbb{P}$ 仅仅是 $[0,1]^K$ 上的一个分布，$\mu$ 从中抽取。

2. 其次，除非另有规定，否则已实现的奖励只能取有限的不同值，而先验 $\mathbb{P}$ 有一个`finite support`，表示为 $\mathcal{F}$ 。然后，我们可以把注意力集中在汤普森抽样的基本概念和论据上，而不是担心积分和概率密度的复杂问题。

3. 第三，对于 $\mathbb{P}$ 支持下的每个平均奖励向量，最佳臂 $a^*$ 是唯一的。这只是为了简单起见：这个假设可以很容易地去掉，但代价是要用稍微繁琐的符号。


## 贝叶斯公式

针对两个随机变量，联合概率分布具有两种分解形式
$$
P(x,y)=P(x|y)P(y)=P(y|x)P(x)
$$
因此，利用上式得到贝叶斯公式

$$
P(c|x)=\frac{P(x|c)P(c)}{P(x)}
$$

通过贝叶斯公式得到贝叶斯决策理论基本思想：

1️⃣ 已知**类条件概率**密度参数表达式$P(x|c)$和**先验概率**$P(c)$

2️⃣ 利用贝叶斯公式转换成**后验概率**

3️⃣ 根据后验概率大小进行决策分类

> **先验概率（prior probability）**：指根据以往经验和分析。在实验或采样前就可以得到的概率。
>
> **后验概率（posterior probability）**：指某件事已经发生，想要计算这件事发生的原因是由某个因素引起的概率。
>
> **类条件概率密度**是，假定x是一个连续随机变量，其分布取决于类别状态，表示成p(x|ω)的形式，这就是“类条件概率密度”函数，即**类别状态为ω时的x的**[概率密度函数](https://baike.baidu.com/item/概率密度函数/5021996)（有时也称为状态条件概率密度）。

## 一、Bayesian update in Bayesian bandits

贝叶斯统计学的一个基本操作是`Bayesian update`：基于新数据，更新先验分布。

### 1.1 术语和符号

固定轮次 $t$ 。算法的前 $t$ 轮数据是一连串的 $action-reward$ 对，称为 $t-history$ 。

$$
H_t = \left({(a_1,r_1), \cdots ,(a_t,r_t) }\right) \in (\mathcal{A}\times \mathbb{R})^t
$$

它是一个随机变量，取决于平均奖励向量 $\mu$、算法和奖励分布（以及三者的随机性）。一个固定的序列

$$
H= \left({(a_1',r_1'), \cdots ,(a_t',r_t') }\right) \in (\mathcal{A}\times \mathbb{R})^t\tag{3.2}
$$

如果满足 $\text{Pr}[H_t=H]>0$ 就被称为 `feasible t-history`, 称这种算法为 `H-consistent`. 其中一个这样的算法，称为`H-induced algorithm`, 在每个回合$s\in [t]$确定性地选择臂$a'_s$。让 $\mathcal{H}_t$ 为成为所有 `feasible t-history`的集合，这个集合是有限的，因为每个奖励只能取有限的值。对于伯努利奖励，$\mathcal{H}_t=(\mathcal{A}\times \{0,1\})^t$ 和一个先验 $\mathbb{P}$ 来说，对所有的臂 $a$ 有 $\text{Pr}[\mu(a)\in (0,1)]=1$

固定一个 `feasible t-history`，我们关心的是条件概率：

$$
\begin{align}
\mathbb{P}_H(\mathcal{M}) := \text{Pr}\left[{ \mu\in\mathcal{M} \ | \  H_t = H }\right],
    \qquad\forall \mathcal{M} \subset [0,1]^K \tag{3.3}
\end{align}
$$

$\mathbb{P}_H$ 被称为 $t$ 轮后的（贝叶斯）后验分布。推导 $\mathbb{P}_H$ 的过程被称为给定 $H$ 的 $\mathbb{P}$ 的`Bayesian update`。


### 1.2 后验不依赖于算法

**引理3.1**： 对于所有`H-consistent`的bandit 算法，分布 $\mathbb{P}_H$ 都是相同的。

算法的动作概率由历史决定，奖励分配由所选择的动作决定。

$data\ history\to choose\ an\ arm\to get\ rewards$

**证明如下**

只需在一个单子集 $\mathcal{M}=\{\tilde{\mu} \}$，任意给定向量 $\tilde{\mu}\in[0,1]^K$的条件下, 证明该定理即可。我们关心条件概率 $\{ \mu=\tilde{\mu}\}$。给定 $r\in\mathbb{R}$，平均奖励 $\tilde{\mu}(a)$ 的奖励分布的概率为 $\mathcal{D}_{\tilde{\mu}(a)}(r)$。

使用归纳法。基础情形是 $t = 0$，我们定义 $0-history$ 为 $H_0=\empty$，这样 $H_0=\empty$ 就是唯一可行的 $0-history$。那么所有的算法都是 $\empty-consistent$ 的，条件概率 $\text{Pr}[\mu = \tilde{\mu} \ | \ H_0=H]$ 就简化为先验概率 $\mathbb{P}(\tilde{\mu})$

$$
\text{Pr}[\mu = \tilde{\mu} \ | \ H_0=H] = \frac{P[ H_0=H \ | \ \mu = \tilde{\mu} ]P(\mu = \tilde{\mu})}{P(H_0=H)}=P(\mu = \tilde{\mu})=\mathbb{P}(\tilde{\mu})
$$

考虑到 $t\geq 1$，把 $H$ 写成某个可行的$(t-1)-history$ $H'$和一个动作-奖励对 $(a,r)$ 的组合，固定一个`H-consistent`的bandit 算法，并让
$$
\pi(a) = \text{Pr}\left[a_t = a|H_{t-1}=H'\right]
$$
为在给定history $H'$ 的情况下，该算法在 $t$ 轮中分配给每个臂 $a$ 的概率。

$$
\begin{aligned}
 \frac{\text{Pr}[\mu = \tilde{\mu} , H_t=H]}{\text{Pr}[H_{t-1}=H']}
    &= \text{Pr}\left[{ \mu = \tilde{\mu} , (a_t, r_t) = (a,r) \ | \  H_{t-1}=H' }\right]\\
    &= \mathbb{P}_{H'}(\tilde{\mu}) \cdot \text{Pr}[ (a_t, r_t) = (a,r) \ | \  \mu = \tilde{\mu} , H_{t-1}=H' ] \\
    &= \mathbb{P}_{H'}(\tilde{\mu}) \\
       &\quad\cdot \text{Pr}\left[{ r_t=r  \ | \  a_t=a , \mu = \tilde{\mu} , H_{t-1}=H' }\right] \\
        &\quad\cdot\text{Pr}\left[{ a_t=a \ | \  \mu = \tilde{\mu} , H_{t-1}=H' }\right] \\
     &= \mathbb{P}_{H'}(\tilde{\mu}) \cdot \mathcal{D}_{\tilde{\mu}(a)}(r) \cdot \pi(a).
\end{aligned}
$$

所以，

$$
\text{Pr}[H_{t}=H]=\pi(a)\cdot \text{Pr}[H_{t-1}=H']\sum_{\tilde{\mu}\in \mathcal{F}}\mathbb{P}_{H'}(\tilde{\mu})\cdot  \mathcal{D}_{\tilde{\mu}(a)}(r)
$$

由此可见，

$$
\begin{aligned}
\mathbb{P}_H(\tilde{\mu})
    = \frac{\Pr[\mu = \tilde{\mu} , H_t=H]}{\Pr[H_t=H]}
    = \frac{\mathbb{P}_{H'}(\tilde{\mu}) \cdot \mathcal{D}_{\tilde{\mu}(a)}(r)}
        {\sum_{\tilde{\mu}\in \mathcal{F}}\mathbb{P}_{H'}(\tilde{\mu}) \cdot \mathcal{D}_{\tilde{\mu}(a)}(r)}
\end{aligned}
$$

通过归纳假设，$\mathbb{P}_H$ 的后验分布不依赖于算法。所以，上面的表达式也不依赖于算法。

由此可见，如果轮次被替换，$\mathbb{P}_H$保持不变。

**推论3.2**：$\mathbb{P}_H = \mathbb{P}_{H'}$ whenever $ H' = \left( \left( a'_{\sigma(t)},\, r'_{\sigma(t)}\right) :\; t\in[T] \right)$ for some permutation $\sigma$ of $[t]$.